In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
engine = create_engine("postgresql:///kc_housing")
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [2]:
def pullsqldata():
    """This function pulls the necessary columns and rows from the PostGRES DB into a Pandas Dataframe in order 
    to continue with our EDA """
    
    engine = create_engine("postgresql:///kc_housing")
    query = """
                SELECT *
                FROM rpsale AS s
                INNER JOIN resbldg AS b ON CONCAT(s.Major,s.Minor) = CONCAT(b.Major, b.Minor)
                INNER JOIN parcel AS p ON CONCAT(s.Major,s.Minor) = CONCAT(p.Major,p.Minor)
                WHERE EXTRACT(YEAR FROM CAST(documentdate AS DATE)) = 2018
                    AND p.proptype = 'R'
                ;"""
    kc_df = pd.read_sql(sql = query, con = engine)
    return kc_df

In [3]:
df = pullsqldata()

In [4]:
def clean_data_initial(df):
    """ This function cleans the housing data by removing outliers and sale price < 10000
    """
    df_clean = df[(df['saleprice']>10000) & (df['saleprice'] <  (2*df['saleprice'].std())+df['saleprice'].mean())]
    return df_clean

In [5]:
df_clean = clean_data_initial(df)

In [6]:
df_clean_drop = df_clean.drop(['documentdate', 
                               'excisetaxnbr', 
                               'recordingnbr', 
                               'volume',
                               'page',
                               'platnbr',
                               'plattype',
                               'platlot',
                               'platblock',
'sellername',
'buyername',
'streetname',
'streettype',
'directionsuffix',
'zipcode',
'buildingnumber',
'major',
'minor',
'bldggradevar',
'sqfthalffloor',
'sqft2ndfloor',
'sqftupperfloor',
'sqftunfinfull',
'sqftunfinhalf',
'sqfttotbasement',
'sqftfinbasement',
'brickstone',
'viewutilization',
'propname',
'platname',
'platlot',
'platblock',
'range',
'township',
'section',
'quartersection',
'area',
'subarea',
'specarea',
'specsubarea',
'levycode',
'districtname',
'currentzoning',
'topography',
'currentusedesignation',
'salewarning', 
'wetland', 
'stream',
'seismichazard',
'landslidehazard',
'address', 
'airportnoise',
'contamination',
'dnrlease',
 'coalminehazard',
 'criticaldrainage',
 'erosionhazard',
 'landfillbuffer',
 'hundredyrfloodplain',
 'steepslopehazard',
 'speciesofconcern',
 'sensitiveareatract',
 'daylightbasement',
 'fraction',
'directionprefix', 'proptype','unbuildable','bldgnbr'], axis=1)

In [7]:
# strips whitespace from othernuisances column, then converts Y to 1 and N to 0
df_clean_drop['othernuisances'] = [x.strip() for x in df_clean['othernuisances']]
df_clean_drop.replace(('Y', 'N'), (1, 0), inplace=True)


//anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
//anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [8]:
df_clean_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28747 entries, 0 to 43738
Data columns (total 83 columns):
saleprice                 28747 non-null float64
propertytype              28747 non-null float64
principaluse              28747 non-null float64
saleinstrument            28747 non-null float64
afforestland              28747 non-null int64
afcurrentuseland          28747 non-null int64
afnonprofituse            28747 non-null int64
afhistoricproperty        28747 non-null int64
salereason                28747 non-null float64
propertyclass             28747 non-null float64
nbrlivingunits            28747 non-null float64
stories                   28747 non-null float64
bldggrade                 28747 non-null float64
sqft1stfloor              28747 non-null float64
sqfttotliving             28747 non-null float64
finbasementgrade          28747 non-null float64
sqftgaragebasement        28747 non-null float64
sqftgarageattached        28747 non-null float64
sqftopenporch    

In [48]:
# creates a DF subset, then combines all fireplaces into a single column "numfp", then drops the other FP columns
feature_eng = df_clean_drop
feature_eng['numfp'] = feature_eng['fpsinglestory']+feature_eng['fpmultistory']+feature_eng['fpfreestanding']+feature_eng['fpadditional']
feature_eng['numbath'] = feature_eng['bathhalfcount'] + feature_eng['bath3qtrcount'] + feature_eng['bathfullcount']
feature_eng = feature_eng.drop(['fpsinglestory', 'fpmultistory', 'fpfreestanding', 'fpadditional','bathhalfcount','bath3qtrcount','bathfullcount'], axis=1)

//anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [51]:
Y = df_clean_drop['saleprice']
X = feature_eng.drop(['saleprice'], axis=1)


In [23]:
lr = LinearRegression()

In [107]:
# worked to strip whitespace
# X['othernuisances'] = [x.strip() for x in X['othernuisances']]

# I dont think the below worked
# pd.Series(map(lambda x: dict(Y=1, N=0)[x],
#               X.othernuisances.values.tolist()), X.index)


In [52]:
select = RFE(lr, n_features_to_select=50)
select = select.fit(X, y= Y.values.ravel())
selected_columns = X.columns[select.support_]

In [53]:
selected_columns

Index(['principaluse', 'saleinstrument', 'afforestland', 'afcurrentuseland',
       'afnonprofituse', 'afhistoricproperty', 'propertyclass',
       'nbrlivingunits', 'stories', 'bldggrade', 'finbasementgrade',
       'heatsource', 'bedrooms', 'pcntnetcondition', 'hbuasifvacant',
       'hbuasimproved', 'watersystem', 'sewersystem', 'restrictiveszshape',
       'inadequateparking', 'mtrainier', 'olympics', 'cascades', 'territorial',
       'seattleskyline', 'pugetsound', 'lakewashington', 'lakesammamish',
       'smalllakerivercreek', 'otherview', 'wfntbank', 'wfntpoorquality',
       'wfntrestrictedaccess', 'wfntaccessrights', 'wfntproximityinfluence',
       'powerlines', 'othernuisances', 'nbrbldgsites', 'adjacentgolffairway',
       'adjacentgreenbelt', 'historicsite', 'nativegrowthprotesmt',
       'easements', 'otherdesignation', 'deedrestrictions',
       'developmentrightspurch', 'waterproblems', 'transpconcurrency',
       'otherproblems', 'numbath'],
      dtype='object')

In [54]:
X_int = sm.add_constant(X)
model = sm.OLS(Y, X_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                     376.2
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        09:43:04   Log-Likelihood:            -3.9910e+05
No. Observations:               28747   AIC:                         7.984e+05
Df Residuals:                   28669   BIC:                         7.990e+05
Df Model:                          77                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   5.348e+06   2.16e+05     24.800      0.000    4.93e+06    5.77e+06
propertytype           -1917.1369    235.764     -8.132      0.000   -2379.246   -1455.028
principaluse           -9.505e+04   2.03e+04     -4.687      0.000   -1.35e+05   -5.53e+04
saleinstrument         -1.194e+04    475.877    -25.097      0.000   -1.29e+04    -1.1e+04
afforestland             3.83e+04    9.9e+04      0.387      0.699   -1.56e+05    2.32e+05
afcurrentuseland       -1.018e+05    5.8e+04     -1.754      0.080   -2.15e+05     1.2e+04
afnonprofituse          -1.85e+05    1.5e+05     -1.234      0.217   -4.79e+05    1.09e+05
afhistoricproperty     -1.163e+05   2.59e+05     -0.449      0.654   -6.25e+05    3.92e+05
salereason             -4196.8799    445.703     -9.416      0.000   -5070.479   -3323.281
propertyclass           -3.57e+04   6830.990     -5.227      0.000   -4.91e+04   -2.23e+04
nbrlivingunits          5.335e+04   1.12e+04      4.756      0.000    3.14e+04    7.53e+04
stories                 3.725e+04   4986.867      7.470      0.000    2.75e+04     4.7e+04
bldggrade               1.631e+05   2428.863     67.146      0.000    1.58e+05    1.68e+05
sqft1stfloor             -39.3340      6.860     -5.734      0.000     -52.780     -25.888
sqfttotliving            140.8220      4.643     30.327      0.000     131.721     149.923
finbasementgrade        2072.0790    687.245      3.015      0.003     725.047    3419.111
sqftgaragebasement       -24.9935     10.805     -2.313      0.021     -46.172      -3.815
sqftgarageattached      -145.1899      8.335    -17.419      0.000    -161.527    -128.853
sqftopenporch            110.1256     14.795      7.444      0.000      81.127     139.124
sqftenclosedporch       -147.4083     37.816     -3.898      0.000    -221.530     -73.287
sqftdeck                   2.7533      1.810      1.521      0.128      -0.795       6.301
heatsystem              4287.5947   1534.498      2.794      0.005    1279.908    7295.282
heatsource             -2356.7654   2757.415     -0.855      0.393   -7761.427    3047.896
bedrooms               -2.562e+04   2178.954    -11.759      0.000   -2.99e+04   -2.14e+04
yrbuilt                -2780.7763     78.443    -35.450      0.000   -2934.528   -2627.024
yrrenovated                2.2441      4.007      0.560      0.575      -5.610      10.098
pcntcomplete            -649.2612    239.516     -2.711      0.007   -1118.725    -179.798
obsolescence            -949.8470    553.586     -1.716      0.086   -2034.902     135.208
pcntnetcondition       -8688.8391   3356.724     -2.588      0.010   -1.53e+04   -2109.503
condition               8898.7110   2469.300      3.604      0.000    4058.767    1.37e+04
addnlcost                 -2.1356      0.603     -3.541      0.000      -3.318  

In [25]:
# sns.heatmap(X.corr())
# X.corr() > .75
## looks for things with a correlation of > 0.5
corr = X.corr() > .55
corr_list = []
for col in corr.columns:
    if corr[col].sum() > 1:
        print(col)
        corr_list.append(col)

bldggrade
sqft1stfloor
sqfttotliving
bedrooms
bathfullcount
olympics
pugetsound
smalllakerivercreek
wfntlocation
wfntfootage
wfntbank
wfntrestrictedaccess
tidelandshoreland


In [170]:
X_experiment = X.drop(corr_list,axis=1)

In [173]:
# select = RFE(lr, n_features_to_select=50)
# select = select.fit(X, y= Y.values.ravel())
# selected_columns = X.columns[select.support_]

X_corr_list = X[corr_list]

X_int = sm.add_constant(X_experiment)
model = sm.OLS(Y, X_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.336
Method:                 Least Squares   F-statistic:                     214.6
Date:                Tue, 03 Dec 2019   Prob (F-statistic):               0.00
Time:                        17:04:54   Log-Likelihood:            -4.0322e+05
No. Observations:               28747   AIC:                         8.066e+05
Df Residuals:                   28678   BIC:                         8.072e+05
Df Model:                          68                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    3.82e+06   2.42e+05     15.814      0.000    3.35e+06    4.29e+06
propertytype           -1982.7762    271.985     -7.290      0.000   -2515.879   -1449.673
principaluse            -7.38e+04   2.34e+04     -3.153      0.002    -1.2e+05   -2.79e+04
saleinstrument         -1.192e+04    547.987    -21.758      0.000    -1.3e+04   -1.08e+04
afforestland             4.14e+04   1.14e+05      0.363      0.717   -1.82e+05    2.65e+05
afcurrentuseland       -1.735e+05   6.69e+04     -2.594      0.009   -3.05e+05   -4.24e+04
afnonprofituse         -1.958e+05   1.73e+05     -1.132      0.258   -5.35e+05    1.43e+05
afhistoricproperty      -2.37e+05   2.99e+05     -0.792      0.429   -8.24e+05     3.5e+05
salereason             -4295.4420    513.740     -8.361      0.000   -5302.397   -3288.487
propertyclass          -2.772e+04   7948.459     -3.488      0.000   -4.33e+04   -1.21e+04
bldgnbr                 1.086e+05   7934.244     13.686      0.000     9.3e+04    1.24e+05
nbrlivingunits          1.165e+05   1.26e+04      9.229      0.000    9.17e+04    1.41e+05
stories                 1.992e+05   3990.762     49.910      0.000    1.91e+05    2.07e+05
finbasementgrade        2.297e+04    630.582     36.432      0.000    2.17e+04    2.42e+04
sqftgaragebasement        97.2956     12.019      8.095      0.000      73.737     120.854
sqftgarageattached       195.8158      8.569     22.853      0.000     179.021     212.611
sqftopenporch            464.5763     16.258     28.576      0.000     432.711     496.442
sqftenclosedporch        -59.0152     43.640     -1.352      0.176    -144.551      26.521
sqftdeck                   9.6947      2.088      4.643      0.000       5.603      13.787
heatsystem               2.02e+04   1752.106     11.531      0.000    1.68e+04    2.36e+04
heatsource             -2.813e+04   3157.742     -8.908      0.000   -3.43e+04   -2.19e+04
bathhalfcount           1.808e+04   3999.864      4.520      0.000    1.02e+04    2.59e+04
bath3qtrcount           6.679e+04   3067.062     21.776      0.000    6.08e+04    7.28e+04
fpsinglestory           5.882e+04   3296.781     17.842      0.000    5.24e+04    6.53e+04
fpfreestanding         -2.611e+04   6919.882     -3.773      0.000   -3.97e+04   -1.25e+04
yrbuilt                -1699.8925     85.139    -19.966      0.000   -1866.768   -1533.017
yrrenovated               34.3544      4.566      7.524      0.000      25.405      43.304
pcntcomplete           -1433.6505    284.119     -5.046      0.000   -1990.536    -876.765
obsolescence           -1208.7545    638.069     -1.894      0.058   -2459.400      41.891
pcntnetcondition       -4681.6226   3872.325     -1.209      0.227   -1.23e+04    2908.316
condition               7756.2240   2836.018      2.735      0.006    2197.496  

In [27]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [28]:
stepwise_selection(X, Y, X)

//anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:45: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


Drop yrrenovated                    with p-value 0.963656
Drop wfntrestrictedaccess           with p-value 0.916593
Drop developmentrightspurch         with p-value 0.860549
Drop tidelandshoreland              with p-value 0.855315
Drop afforestland                   with p-value 0.831196
Drop otherproblems                  with p-value 0.777736
Drop afhistoricproperty             with p-value 0.597244
Drop pcntunusable                   with p-value 0.577374
Drop wfntpoorquality                with p-value 0.532585
Drop access                         with p-value 0.459047
Drop watersystem                    with p-value 0.412986
Drop wfntfootage                    with p-value 0.298963
Drop wfntproximityinfluence         with p-value 0.269528
Drop trafficnoise                   with p-value 0.261226
Drop otherview                      with p-value 0.232443
Drop afnonprofituse                 with p-value 0.206674
Drop pugetsound                     with p-value 0.192368
Drop sqftdeck 

['propertytype',
 'principaluse',
 'saleinstrument',
 'salereason',
 'propertyclass',
 'nbrlivingunits',
 'stories',
 'bldggrade',
 'sqft1stfloor',
 'sqfttotliving',
 'sqftgarageattached',
 'sqftopenporch',
 'sqftenclosedporch',
 'heatsystem',
 'bedrooms',
 'bathhalfcount',
 'bath3qtrcount',
 'bathfullcount',
 'yrbuilt',
 'pcntcomplete',
 'pcntnetcondition',
 'condition',
 'addnlcost',
 'hbuasifvacant',
 'hbuasimproved',
 'presentuse',
 'sqftlot',
 'sewersystem',
 'streetsurface',
 'restrictiveszshape',
 'inadequateparking',
 'mtrainier',
 'olympics',
 'cascades',
 'territorial',
 'seattleskyline',
 'lakewashington',
 'lakesammamish',
 'wfntbank',
 'wfntaccessrights',
 'lotdepthfactor',
 'powerlines',
 'othernuisances',
 'nbrbldgsites',
 'adjacentgolffairway',
 'adjacentgreenbelt',
 'historicsite',
 'nativegrowthprotesmt',
 'easements',
 'otherdesignation',
 'deedrestrictions',
 'waterproblems',
 'numfp']

In [29]:
X_sub = X[['propertytype',
 'principaluse',
 'saleinstrument',
 'salereason',
 'propertyclass',
 'nbrlivingunits',
 'stories',
 'bldggrade',
 'sqft1stfloor',
 'sqfttotliving',
 'sqftgarageattached',
 'sqftopenporch',
 'sqftenclosedporch',
 'heatsystem',
 'bedrooms',
 'bathhalfcount',
 'bath3qtrcount',
 'bathfullcount',
 'yrbuilt',
 'pcntcomplete',
 'pcntnetcondition',
 'condition',
 'addnlcost',
 'hbuasifvacant',
 'hbuasimproved',
 'presentuse',
 'sqftlot',
 'sewersystem',
 'streetsurface',
 'restrictiveszshape',
 'inadequateparking',
 'mtrainier',
 'olympics',
 'cascades',
 'territorial',
 'seattleskyline',
 'lakewashington',
 'lakesammamish',
 'wfntbank',
 'wfntaccessrights',
 'lotdepthfactor',
 'powerlines',
 'othernuisances',
 'nbrbldgsites',
 'adjacentgolffairway',
 'adjacentgreenbelt',
 'historicsite',
 'nativegrowthprotesmt',
 'easements',
 'otherdesignation',
 'deedrestrictions',
 'waterproblems',
 'numfp']]

ValueError: Invalid broadcasting comparison [['propertytype', 'principaluse', 'saleinstrument', 'salereason', 'propertyclass', 'nbrlivingunits', 'stories', 'bldggrade', 'sqft1stfloor', 'sqfttotliving', 'sqftgarageattached', 'sqftopenporch', 'sqftenclosedporch', 'heatsystem', 'bedrooms', 'bathhalfcount', 'bath3qtrcount', 'bathfullcount', 'yrbuilt', 'pcntcomplete', 'pcntnetcondition', 'condition', 'addnlcost', 'hbuasifvacant', 'hbuasimproved', 'presentuse', 'sqftlot', 'sewersystem', 'streetsurface', 'restrictiveszshape', 'inadequateparking', 'mtrainier', 'olympics', 'cascades', 'territorial', 'seattleskyline', 'lakewashington', 'lakesammamish', 'wfntbank', 'wfntaccessrights', 'lotdepthfactor', 'powerlines', 'othernuisances', 'nbrbldgsites', 'adjacentgolffairway', 'adjacentgreenbelt', 'historicsite', 'nativegrowthprotesmt', 'easements', 'otherdesignation', 'deedrestrictions', 'waterproblems', 'numfp']] with block values

In [65]:
X_int = sm.add_constant(X_sub['sqfttotliving'])
model = sm.OLS(Y, X_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                 1.199e+04
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        09:57:41   Log-Likelihood:            -4.0412e+05
No. Observations:               28747   AIC:                         8.083e+05
Df Residuals:                   28745   BIC:                         8.083e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            2.5e+05   4626.487     54.041      0.000    2.41e+05    2.59e+05
sqfttotliving   221.9773      2.027    109.512      0.000     218.004     225.950
==============================================================================
Omnibus:                     2978.402   Durbin-Watson:                   0.867
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6914.952
Skew:                           0.630   Prob(JB):                         0.00
Kurtosis:                       5.046   Cond. No.                     5.81e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.81e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [56]:
df_clean_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28747 entries, 0 to 43738
Data columns (total 85 columns):
saleprice                 28747 non-null float64
propertytype              28747 non-null float64
principaluse              28747 non-null float64
saleinstrument            28747 non-null float64
afforestland              28747 non-null int64
afcurrentuseland          28747 non-null int64
afnonprofituse            28747 non-null int64
afhistoricproperty        28747 non-null int64
salereason                28747 non-null float64
propertyclass             28747 non-null float64
nbrlivingunits            28747 non-null float64
stories                   28747 non-null float64
bldggrade                 28747 non-null float64
sqft1stfloor              28747 non-null float64
sqfttotliving             28747 non-null float64
finbasementgrade          28747 non-null float64
sqftgaragebasement        28747 non-null float64
sqftgarageattached        28747 non-null float64
sqftopenporch    

In [75]:
## ANOVA Shit

formula = 'saleprice ~ C(heatsource) + C(bldggrade) + C(nbrlivingunits) + seattleskyline + smalllakerivercreek + lakesammamish + lakewashington + pugetsound + territorial + cascades + olympics + mtrainier + otherview'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)

In [76]:
table

,sum_sq,df,F,PR(>F)
C(heatsource),1.876365e+13,7.0,5.683768,1.392676e-06
C(bldggrade),2.796409e+15,13.0,456.115097,0.000000e+00
C(nbrlivingunits),1.477779e+13,2.0,15.667387,1.578259e-07
seattleskyline,1.233736e+13,1.0,26.160098,3.155744e-07
smalllakerivercreek,2.524477e+11,1.0,0.535289,4.643956e-01
lakesammamish,9.223194e+11,1.0,1.955683,1.619829e-01
lakewashington,1.417219e+14,1.0,300.506647,4.287209e-67
pugetsound,1.934203e+11,1.0,0.410128,5.219081e-01
territorial,1.048515e+12,1.0,2.223267,1.359524e-01
cascades,1.261096e+13,1.0,26.740229,2.337493e-07


In [73]:
f2 = 'saleprice ~ sqfttotliving'
model = ols(formula=f2, data=df_clean_drop).fit()

In [74]:
df_clean_drop

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                 1.199e+04
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        10:21:43   Log-Likelihood:            -4.0412e+05
No. Observations:               28747   AIC:                         8.083e+05
Df Residuals:                   28745   BIC:                         8.083e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        2.5e+05   4626.487     54.041      0.000    2.41e+05    2.59e+05
sqfttotliving   221.9773      2.027    109.512      0.000     218.004     225.950
==============================================================================
Omnibus:                     2978.402   Durbin-Watson:                   0.867
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6914.952
Skew:                           0.630   Prob(JB):                         0.00
Kurtosis:                       5.046   Cond. No.                     5.81e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.81e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""